In [2]:
# importing QISKit
from qiskit import BasicAer, IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import ParameterVector, Parameter
# imports needed for VQC
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.datasets import wine
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap, NLocal
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import COBYLA,SPSA
# useful math functions
from math import pi

In [1]:
!pip install qiskit
!pip install qiskit_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 4.8 MB/s 
     |████████████████████████████████| 19.2 MB 7.1 MB/s 
     |████████████████████████████████| 240 kB 49.3 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 106 kB 38.7 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 37.5 MB 1.4 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 943 kB 57.5 MB/s 
     |████████████████████████████████| 4.1 MB 37.2 MB/s 
     |████████████████████████████████| 1.9 MB 47.9 MB/s 
     |████████████████████████████████| 112 kB 48.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.2-py3-none-any.whl size=12224 sha256=df4fbdecf3ff8ed3f8c12145cf5d920e167cbb0c02b4ba86d41081917c43f662
  Stored in directory: /root/.cache/pip/wheels/30/dc/90/0ee55

In [3]:
import numpy as np
train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')
test_data = np.load('test_data.npy')
test_labels = np.load('test_labels.npy')

In [4]:
train_data_neighbours = train_data[:,2]
train_neighbour_bits = []
for i in range(len(train_data)):
  if train_data_neighbours[i]<=10:
    train_neighbour_bits.append('00')
  elif train_data_neighbours[i]<= 20 and train_data_neighbours[i]>10:
    train_neighbour_bits.append('01')
  elif train_data_neighbours[i]<=100 and train_data_neighbours[i]>20:
    train_neighbour_bits.append('10')
  else:
    train_neighbour_bits.append('11')
test_data_neighbours = test_data[:,2]
test_neighbour_bits = []
for i in range(len(test_data)):
  if test_data_neighbours[i]<=10:
    test_neighbour_bits.append('00')
  elif test_data_neighbours[i]<=20 and test_data_neighbours[i]>10:
    test_neighbour_bits.append('01')
  elif test_data_neighbours[i]<=100 and test_data_neighbours[i]>20:
    test_neighbour_bits.append('10')
  else:
    test_neighbour_bits.append('11')

train_feas2_bits = np.array(train_neighbour_bits)
test_feas2_bits = np.array(test_neighbour_bits)

In [5]:
train_data_neighbours = train_data[:,6]
train_neighbour_bits = []
for i in range(len(train_data)):
  if train_data_neighbours[i]<=10:
    train_neighbour_bits.append('00')
  elif train_data_neighbours[i]<= 20 and train_data_neighbours[i]>10:
    train_neighbour_bits.append('01')
  elif train_data_neighbours[i]<=100 and train_data_neighbours[i]>20:
    train_neighbour_bits.append('10')
  else:
    train_neighbour_bits.append('11')
test_data_neighbours = test_data[:,6]
test_neighbour_bits = []
for i in range(len(test_data)):
  if test_data_neighbours[i]<=10:
    test_neighbour_bits.append('00')
  elif test_data_neighbours[i]<=20 and test_data_neighbours[i]>10:
    test_neighbour_bits.append('01')
  elif test_data_neighbours[i]<=100 and test_data_neighbours[i]>20:
    test_neighbour_bits.append('10')
  else:
    test_neighbour_bits.append('11')

train_feas6_bits = np.array(train_neighbour_bits)
test_feas6_bits = np.array(test_neighbour_bits)

In [6]:
train_data_neighbours = train_data[:,5]
train_neighbour_bits = []
for i in range(len(train_data)):
  if train_data_neighbours[i]<=10:
    train_neighbour_bits.append('00')
  elif train_data_neighbours[i]<= 25 and train_data_neighbours[i]>10:
    train_neighbour_bits.append('01')
  elif train_data_neighbours[i]<=100 and train_data_neighbours[i]>25:
    train_neighbour_bits.append('10')
  else:
    train_neighbour_bits.append('11')
test_data_neighbours = test_data[:,5]
test_neighbour_bits = []
for i in range(len(test_data)):
  if test_data_neighbours[i]<=10:
    test_neighbour_bits.append('00')
  elif test_data_neighbours[i]<=25 and test_data_neighbours[i]>10:
    test_neighbour_bits.append('01')
  elif test_data_neighbours[i]<=100 and test_data_neighbours[i]>25:
    test_neighbour_bits.append('10')
  else:
    test_neighbour_bits.append('11')

train_feas5_bits = np.array(train_neighbour_bits)
test_feas5_bits = np.array(test_neighbour_bits)

In [7]:
train_data_bits = np.column_stack((train_feas2_bits,train_feas5_bits,train_feas6_bits))
test_data_bits = np.column_stack((test_feas2_bits,test_feas5_bits,test_feas6_bits))

In [8]:
from math import sqrt, cos, acos
theta = acos(sqrt(0.5 + sqrt(3.0)/6.0))
num_features = 3
train_angles1 = np.zeros((len(train_data_bits),3*num_features))
for i in range(len(train_data)):
  for j in range(num_features):
    if train_data_bits[i][j] == '00':
      train_angles1[i][j] = 2*theta
      train_angles1[i][j+1] = pi/4
      train_angles1[i][j+2] = -pi/4
    elif train_data_bits[i][j] == '10':
      train_angles1[i][j] = pi-2*theta
      train_angles1[i][j+1] = 3*pi/4
      train_angles1[i][j+2] = -3*pi/4
    elif train_data_bits[i][j] == '11':
      train_angles1[i][j] = 2*theta
      train_angles1[i][j+1] = -3*pi/4
      train_angles1[i][j+2] = 3*pi/4
    else:
      train_angles1[i][j] = pi-2*theta
      train_angles1[i][j+1] = -pi/4
      train_angles1[i][j+2] = pi/4

test_angles1 = np.zeros((len(test_data_bits),3*num_features))
for i in range(len(test_data)):
  for j in range(num_features):
    if test_data_bits[i][j] == '00':
      test_angles1[i][j] = 2*theta
      test_angles1[i][j+1] = pi/4
      test_angles1[i][j+2] = -pi/4
    elif test_data_bits[i][j] == '10':
      test_angles1[i][j] = pi-2*theta
      test_angles1[i][j+1] = 3*pi/4
      test_angles1[i][j+2] = -3*pi/4
    elif test_data_bits[i][j] == '11':
      test_angles1[i][j] = 2*theta
      test_angles1[i][j+1] = -3*pi/4
      test_angles1[i][j+2] = 3*pi/4
    else:
      test_angles1[i][j] = pi-2*theta
      test_angles1[i][j+1] = -pi/4
      test_angles1[i][j+2] = pi/4

In [9]:
from sklearn.metrics import classification_report, confusion_matrix
train_labels_svm = train_labels[:,0]
test_labels_svm = test_labels[:,0]

In [10]:
param2 = ParameterVector('x',3*num_features)
fm = QuantumCircuit(num_features)
fm.u3(param2[0], param2[1], param2[2], 0)
fm.u3(param2[3], param2[4], param2[5], 1)
fm.u3(param2[6], param2[7], param2[8], 2)
fm.draw()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-pa

┌────────────────────┐
q_0: ┤ U3(x[0],x[1],x[2]) ├
     ├────────────────────┤
q_1: ┤ U3(x[3],x[4],x[5]) ├
     ├────────────────────┤
q_2: ┤ U3(x[6],x[7],x[8]) ├
     └────────────────────┘

#Ansatz1

In [11]:
import ansatz19 as ansatz_19
from time import time
seed = 1005
ansatz1 = ansatz_19.get_ansatz(1,3,2)
vqc_1 = VQC(feature_map=fm,
            ansatz=ansatz1,
            optimizer=COBYLA(maxiter=100),
            quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            seed_simulator=seed, seed_transpiler= seed)
            )
start_time_train = time()
vqc_1.fit(train_angles1,train_labels)
end_time_train = time()
y_1 = vqc_1.predict(test_angles1)
end_time_test = time()
label_1 = np.delete(y_1,1,1)
print('training time: ',end_time_train-start_time_train)
print('testing time: ',end_time_test-end_time_train)
print(classification_report(test_labels_svm,label_1))
print(confusion_matrix(test_labels_svm,label_1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The quantum_instance argument is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the sampler argument.
  if __name__ == '__main__':


training time:  421.3069324493408
testing time:  268.26406264305115
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     10000
           1       0.51      0.46      0.48      1000

    accuracy                           0.91     11000
   macro avg       0.73      0.71      0.72     11000
weighted avg       0.91      0.91      0.91     11000

[[9553  447]
 [ 539  461]]


#Ansatz 2

In [12]:
ansatz2 = ansatz_19.get_ansatz(2,3,2)
vqc_1 = VQC(feature_map=fm,
            ansatz=ansatz2,
            optimizer=COBYLA(maxiter=100),
            quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            seed_simulator=seed, seed_transpiler= seed)
            )
start_time_train = time()
vqc_1.fit(train_angles1,train_labels)
end_time_train = time()
y_1 = vqc_1.predict(test_angles1)
end_time_test = time()
label_1 = np.delete(y_1,1,1)
print('training time: ',end_time_train-start_time_train)
print('testing time: ',end_time_test-end_time_train)
print(classification_report(test_labels_svm,label_1))
print(confusion_matrix(test_labels_svm,label_1))

training time:  428.7924144268036
testing time:  289.4005835056305
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     10000
           1       0.68      0.52      0.59      1000

    accuracy                           0.93     11000
   macro avg       0.82      0.75      0.78     11000
weighted avg       0.93      0.93      0.93     11000

[[9760  240]
 [ 481  519]]


#Ansatz3

In [13]:
ansatz3 = ansatz_19.get_ansatz(3,3,2)
vqc_1 = VQC(feature_map=fm,
            ansatz=ansatz3,
            optimizer=COBYLA(maxiter=100),
            quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            seed_simulator=seed, seed_transpiler= seed)
            )
start_time_train = time()
vqc_1.fit(train_angles1,train_labels)
end_time_train = time()
y_1 = vqc_1.predict(test_angles1)
end_time_test = time()
label_1 = np.delete(y_1,1,1)
print('training time: ',end_time_train-start_time_train)
print('testing time: ',end_time_test-end_time_train)
print(classification_report(test_labels_svm,label_1))
print(confusion_matrix(test_labels_svm,label_1))

training time:  588.2976064682007
testing time:  332.1376066207886
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     10000
           1       0.51      0.46      0.48      1000

    accuracy                           0.91     11000
   macro avg       0.73      0.71      0.72     11000
weighted avg       0.91      0.91      0.91     11000

[[9553  447]
 [ 539  461]]


#Ansatz4

In [14]:
ansatz4 = ansatz_19.get_ansatz(4,3,2)
vqc_1 = VQC(feature_map=fm,
            ansatz=ansatz4,
            optimizer=COBYLA(maxiter=100),
            quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            seed_simulator=seed, seed_transpiler= seed)
            )
start_time_train = time()
vqc_1.fit(train_angles1,train_labels)
end_time_train = time()
y_1 = vqc_1.predict(test_angles1)
end_time_test = time()
label_1 = np.delete(y_1,1,1)
print('training time: ',end_time_train-start_time_train)
print('testing time: ',end_time_test-end_time_train)
print(classification_report(test_labels_svm,label_1))
print(confusion_matrix(test_labels_svm,label_1))

training time:  609.5610165596008
testing time:  321.30195474624634
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     10000
           1       0.66      0.43      0.52      1000

    accuracy                           0.93     11000
   macro avg       0.80      0.70      0.74     11000
weighted avg       0.92      0.93      0.92     11000

[[9781  219]
 [ 573  427]]
